In [1]:
import pandas as pd
import numpy as np
# чтоб картинки рисовались в тетрадке
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (10, 8)

from multiprocessing import Pool
import time

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_colwidth', 1000)

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')


from pylab import rcParams

from sklearn.cluster import KMeans

